<a href="https://colab.research.google.com/github/DanielTorres355/Prueba-1/blob/main/TP1/prueba_bi_publisher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#subir archivo excel
from google.colab import files
uploaded = files.upload()


#Leer archivo excel
import pandas as pd

# Reemplaza con el nombre de tu archivo
df = pd.read_excel("BOGOTA_PRIME.xlsx")  # Usa openpyxl por defecto
df.head()  # Muestra las primeras filas
#Si el archivo tiene varias hojas, puedes especificar una:

#python
#Copiar
#Editar

df = pd.read_excel("BOGOTA_PRIME.xlsx", sheet_name="Sheet1")


# Cambiar un valor específico
df.loc[0, 'Nombre'] = "Juan"

# Agregar una nueva fila
nueva_fila = {'Nombre': 'Ana', 'Edad': 28}
df = df.append(nueva_fila, ignore_index=True)

# Eliminar filas
df = df.drop(1)  # Elimina la fila con índice 1


#Guardaer los datos en un nuevo excel
df.to_excel("archivo_editado.xlsx", index=False)

#Descargar el archivo
#files.download("archivo_editado.xlsx")

In [ ]:
#1
from google.colab import files
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets
from itertools import permutations

# Paso 1: Subir archivo Excel
uploaded = files.upload()

for nombre_archivo in uploaded.keys():
    # Leer archivo con header correcto en fila 2
    df = pd.read_excel(io.BytesIO(uploaded[nombre_archivo]), header=2)

# Paso 2: Limpiar los datos
df = df[df['Asignatura'].notna()]  # Filtrar filas vacías

# Paso 3: Transformar columnas de días a filas
dias_map = {
    'Lun': 'Lunes',
    'Mar': 'Martes',
    'Mier': 'Miércoles',
    'Jue': 'Jueves',
    'Vier': 'Viernes',
    'Sab': 'Sábado'
}

df_largo = df.melt(
    id_vars=['Asignatura', 'Hora Ini', 'Hora Fin', 'Salon', 'Campus', 'Total Inscritos', 'Total Cupos'],
    value_vars=dias_map.keys(),
    var_name='Día Abrev',
    value_name='Activo'
)

df_largo = df_largo[df_largo['Activo'] == 'Y']
df_largo['Día'] = df_largo['Día Abrev'].map(dias_map)
df_largo = df_largo.drop(columns=['Día Abrev', 'Activo'])

# Paso 4: Mostrar resumen de materias
resumen = df_largo.groupby('Asignatura').agg({
    'Día': lambda x: ', '.join(sorted(x.unique())),
    'Hora Ini': 'min',
    'Hora Fin': 'max',
    'Campus': 'first'
}).reset_index()

print("📋 Materias disponibles:")
display(resumen)

# Paso 5: Widget para seleccionar materias
materias = sorted(df_largo['Asignatura'].unique())
selector_materias = widgets.SelectMultiple(
    options=materias,
    description='Materias:',
    layout=widgets.Layout(width='50%'),
    rows=10
)
display(selector_materias)

# Paso 6: Función de validación y generación de horarios
def hay_conflicto(horario):
    horario['Hora Ini'] = pd.to_datetime(horario['Hora Ini'].astype(str), format='%H:%M').dt.time
    horario['Hora Fin'] = pd.to_datetime(horario['Hora Fin'].astype(str), format='%H:%M').dt.time

    for dia in horario['Día'].unique():
        clases_dia = horario[horario['Día'] == dia].sort_values('Hora Ini')
        for i in range(len(clases_dia) - 1):
            h1_fin = clases_dia.iloc[i]['Hora Fin']
            h2_ini = clases_dia.iloc[i + 1]['Hora Ini']
            if h2_ini < h1_fin:
                return True
    return False

def mostrar_horario(horario):
    dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado']
    horario = horario.sort_values(by=['Día', 'Hora Ini'])
    for dia in dias:
        clases = horario[horario['Día'] == dia]
        if not clases.empty:
            print(f"\n📅 {dia}")
            for _, row in clases.iterrows():
                print(f"🕒 {row['Hora Ini']} - {row['Hora Fin']}: {row['Asignatura']} (Salón: {row['Salon']})")

def generar_opciones_horarios(df_largo, materias_seleccionadas):
    df_filtrado = df_largo[df_largo['Asignatura'].isin(materias_seleccionadas)]
    opciones = []
    clases_por_materia = df_filtrado.groupby('Asignatura')

    clases_dict = {m: g for m, g in clases_por_materia}
    clases_combinadas = list(permutations(clases_dict.values(), len(materias_seleccionadas)))

    for grupo in clases_combinadas:
        seleccion = pd.concat(grupo).drop_duplicates()
        if not hay_conflicto(seleccion):
            opciones.append(seleccion)
        if len(opciones) == 3:
            break

    return opciones

# Paso 7: Botón para generar horarios
boton_generar = widgets.Button(description="Generar horarios", button_style='success')
output = widgets.Output()

def on_boton_generar_clicked(b):
    clear_output(wait=True)
    display(selector_materias, boton_generar, output)
    materias_seleccionadas = list(selector_materias.value)

    if not materias_seleccionadas:
        print("⚠️ Por favor selecciona al menos una materia.")
        return

    opciones = generar_opciones_horarios(df_largo, materias_seleccionadas)

    if not opciones:
        print("❌ No se encontraron combinaciones sin cruce de horario.")
    else:
        for idx, horario in enumerate(opciones):
            print(f"\n🔹 Opción {idx + 1}")
            mostrar_horario(horario)

boton_generar.on_click(on_boton_generar_clicked)

display(boton_generar, output)

#---------------------------------------------------------
import tempfile
import os

# Para almacenar las opciones de horario globalmente (evitar que se sobrescriban)
opciones_globales = []

def crear_boton_guardar(idx, horario_df):
    boton = widgets.Button(description=f"💾 Guardar Opción {idx + 1}", button_style='info')

    def on_click_guardar(b):
        with tempfile.NamedTemporaryFile(delete=False, suffix=".xlsx") as tmp:
            horario_df.to_excel(tmp.name, index=False)
            tmp.flush()
            with open(tmp.name, "rb") as f:
                content = f.read()
            # Descargar archivo en el entorno de Colab
            files.download(tmp.name)

    boton.on_click(on_click_guardar)
    return boton

def on_boton_generar_clicked(b):
    clear_output(wait=True)
    display(selector_materias, boton_generar, output)
    materias_seleccionadas = list(selector_materias.value)

    if not materias_seleccionadas:
        print("⚠️ Por favor selecciona al menos una materia.")
        return

    opciones_globales.clear()
    opciones = generar_opciones_horarios(df_largo, materias_seleccionadas)

    if not opciones:
        print("❌ No se encontraron combinaciones sin cruce de horario.")
    else:
        for idx, horario in enumerate(opciones):
            print(f"\n🔹 Opción {idx + 1}")
            mostrar_horario(horario)
            opciones_globales.append(horario.copy())  # Guardar la opción
            display(crear_boton_guardar(idx, horario))


In [ ]:
#2 📚 Importar librerías
from google.colab import files
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime

# 📂 Paso 1: Subir archivo Excel
uploaded = files.upload()
for nombre_archivo in uploaded.keys():
    df = pd.read_excel(io.BytesIO(uploaded[nombre_archivo]), header=2)

# 🧹 Paso 2: Limpiar y transformar los datos
df = df[df['Asignatura'].notna()]

dias_map = {
    'Lun': 'Lunes',
    'Mar': 'Martes',
    'Mier': 'Miércoles',
    'Jue': 'Jueves',
    'Vier': 'Viernes',
    'Sab': 'Sábado'
}

df_largo = df.melt(
    id_vars=['Asignatura', 'Hora Ini', 'Hora Fin', 'Salon', 'Campus', 'Total Inscritos', 'Total Cupos'],
    value_vars=dias_map.keys(),
    var_name='Día Abrev',
    value_name='Activo'
)

df_largo = df_largo[df_largo['Activo'] == 'Y']
df_largo['Día'] = df_largo['Día Abrev'].map(dias_map)
df_largo = df_largo.drop(columns=['Día Abrev', 'Activo'])

df_largo['Total Inscritos'] = pd.to_numeric(df_largo['Total Inscritos'], errors='coerce')
df_largo['Total Cupos'] = pd.to_numeric(df_largo['Total Cupos'], errors='coerce')
df_largo['% Ocupación'] = (df_largo['Total Inscritos'] / df_largo['Total Cupos']) * 100

df_largo['Hora Ini'] = pd.to_datetime(df_largo['Hora Ini'].astype(str), format='%H:%M').dt.time
df_largo['Hora Fin'] = pd.to_datetime(df_largo['Hora Fin'].astype(str), format='%H:%M').dt.time

# 📋 Paso 3: Mostrar resumen general de materias
resumen = df_largo.groupby('Asignatura').agg({
    'Día': lambda x: ', '.join(sorted(x.unique())),
    'Hora Ini': lambda x: min(x).strftime('%H:%M'),
    'Hora Fin': lambda x: max(x).strftime('%H:%M'),
    'Total Inscritos': 'first',
    'Total Cupos': 'first',
    '% Ocupación': 'first'
}).reset_index()

print("📋 Oferta académica disponible:")
display(resumen)

# ✅ Paso 4: Widget para seleccionar materias
materias = sorted(df_largo['Asignatura'].unique())
selector_materias = widgets.SelectMultiple(
    options=materias,
    description='Materias:',
    layout=widgets.Layout(width='50%'),
    rows=10
)

boton_generar = widgets.Button(description="Generar horario", button_style='success')
output = widgets.Output()

# 🗓️ Paso 5: Funciones de visualización
def hora_a_minutos(t):
    return t.hour * 60 + t.minute

def color_por_ocupacion(pct):
    if pct < 50:
        return 'green'
    elif pct <= 90:
        return 'gold'
    else:
        return 'red'

dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado']
mapa_dias = {dia: i for i, dia in enumerate(dias_semana)}

def mostrar_calendario(horario_df):
    fig, ax = plt.subplots(figsize=(14, 8))

    for _, row in horario_df.iterrows():
        dia_idx = mapa_dias[row['Día']]
        ini_min = hora_a_minutos(row['Hora Ini'])
        fin_min = hora_a_minutos(row['Hora Fin'])
        duracion = fin_min - ini_min

        rect = patches.Rectangle(
            (dia_idx, ini_min),
            width=0.9,
            height=duracion,
            facecolor=color_por_ocupacion(row['% Ocupación']),
            edgecolor='black',
            alpha=0.8
        )
        ax.add_patch(rect)
        ax.text(
            dia_idx + 0.05, ini_min + duracion / 2,
            f"{row['Asignatura']}\n{row['Hora Ini'].strftime('%H:%M')} - {row['Hora Fin'].strftime('%H:%M')}",
            fontsize=8,
            verticalalignment='center',
            color='black'
        )

    ax.set_xlim(0, len(dias_semana))
    ax.set_ylim(420, 1320)
    ax.set_xticks(range(len(dias_semana)))
    ax.set_xticklabels(dias_semana)
    ax.set_yticks(range(420, 1321, 60))
    ax.set_yticklabels([f"{h}:00" for h in range(7, 23)])
    ax.set_xlabel("Día")
    ax.set_ylabel("Hora")
    ax.set_title("🗓️ Horario personalizado del estudiante")
    ax.grid(True, axis='y', linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

# ▶️ Paso 6: Lógica para generar horario
def on_boton_generar_clicked(b):
    with output:
        clear_output(wait=True)
        seleccion = list(selector_materias.value)
        if not seleccion:
            print("⚠️ Por favor selecciona al menos una materia.")
            return
        df_seleccion = df_largo[df_largo['Asignatura'].isin(seleccion)]
        mostrar_calendario(df_seleccion)

boton_generar.on_click(on_boton_generar_clicked)

# ▶️ Mostrar widgets
display(selector_materias, boton_generar, output)

In [ ]:
#3 📚 Importar librerías
from google.colab import files
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime

# 📂 Paso 1: Subir archivo Excel
uploaded = files.upload()
for nombre_archivo in uploaded.keys():
    df = pd.read_excel(io.BytesIO(uploaded[nombre_archivo]), header=2)

# 🧹 Paso 2: Limpiar y transformar los datos
df = df[df['Asignatura'].notna()]

dias_map = {
    'Lun': 'Lunes',
    'Mar': 'Martes',
    'Mier': 'Miércoles',
    'Jue': 'Jueves',
    'Vier': 'Viernes',
    'Sab': 'Sábado'
}

df_largo = df.melt(
    id_vars=['Asignatura', 'Hora Ini', 'Hora Fin', 'Salon', 'Campus', 'Total Inscritos', 'Total Cupos'],
    value_vars=dias_map.keys(),
    var_name='Día Abrev',
    value_name='Activo'
)

df_largo = df_largo[df_largo['Activo'] == 'Y']
df_largo['Día'] = df_largo['Día Abrev'].map(dias_map)
df_largo = df_largo.drop(columns=['Día Abrev', 'Activo'])

df_largo['Total Inscritos'] = pd.to_numeric(df_largo['Total Inscritos'], errors='coerce')
df_largo['Total Cupos'] = pd.to_numeric(df_largo['Total Cupos'], errors='coerce')
df_largo['% Ocupación'] = (df_largo['Total Inscritos'] / df_largo['Total Cupos']) * 100

df_largo['Hora Ini'] = pd.to_datetime(df_largo['Hora Ini'].astype(str), format='%H:%M').dt.time
df_largo['Hora Fin'] = pd.to_datetime(df_largo['Hora Fin'].astype(str), format='%H:%M').dt.time

# 📋 Paso 3: Mostrar resumen general de materias
resumen = df_largo.groupby('Asignatura').agg({
    'Día': lambda x: ', '.join(sorted(x.unique())),
    'Hora Ini': lambda x: min(x).strftime('%H:%M'),
    'Hora Fin': lambda x: max(x).strftime('%H:%M'),
    'Total Inscritos': 'first',
    'Total Cupos': 'first',
    '% Ocupación': 'first'
}).reset_index()

print("📋 Oferta académica disponible:")
display(resumen)

# ✅ Paso 4: Widget para seleccionar materias
materias = sorted(df_largo['Asignatura'].unique())
selector_materias = widgets.SelectMultiple(
    options=materias,
    description='Materias:',
    layout=widgets.Layout(width='50%'),
    rows=10
)

boton_generar = widgets.Button(description="Generar horario", button_style='success')
output = widgets.Output()

# 🗓️ Paso 5: Funciones de visualización
def hora_a_minutos(t):
    return t.hour * 60 + t.minute

def color_por_ocupacion(pct):
    if pct < 50:
        return 'green'
    elif pct <= 90:
        return 'gold'
    else:
        return 'red'

dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado']
mapa_dias = {dia: i for i, dia in enumerate(dias_semana)}

def mostrar_calendario(horario_df):
    fig, ax = plt.subplots(figsize=(14, 8))

    for _, row in horario_df.iterrows():
        dia_idx = mapa_dias[row['Día']]
        ini_min = hora_a_minutos(row['Hora Ini'])
        fin_min = hora_a_minutos(row['Hora Fin'])
        duracion = fin_min - ini_min

        rect = patches.Rectangle(
            (dia_idx, ini_min),
            width=0.9,
            height=duracion,
            facecolor=color_por_ocupacion(row['% Ocupación']),
            edgecolor='black',
            alpha=0.8
        )
        ax.add_patch(rect)
        ax.text(
            dia_idx + 0.05, ini_min + duracion / 2,
            f"{row['Asignatura']}\n{row['Hora Ini'].strftime('%H:%M')} - {row['Hora Fin'].strftime('%H:%M')}",
            fontsize=8,
            verticalalignment='center',
            color='black'
        )

    ax.set_xlim(0, len(dias_semana))
    ax.set_ylim(420, 1320)
    ax.set_xticks(range(len(dias_semana)))
    ax.set_xticklabels(dias_semana)
    ax.set_yticks(range(420, 1321, 60))
    ax.set_yticklabels([f"{h}:00" for h in range(7, 23)])
    ax.set_xlabel("Día")
    ax.set_ylabel("Hora")
    ax.set_title("🗓️ Horario personalizado del estudiante")
    ax.grid(True, axis='y', linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

# ▶️ Paso 6: Lógica para generar horario
def on_boton_generar_clicked(b):
    with output:
        clear_output(wait=True)
        seleccion = list(selector_materias.value)
        if not seleccion:
            print("⚠️ Por favor selecciona al menos una materia.")
            return
        df_seleccion = df_largo[df_largo['Asignatura'].isin(seleccion)]
        mostrar_calendario(df_seleccion)

boton_generar.on_click(on_boton_generar_clicked)

# ▶️ Mostrar widgets
display(selector_materias, boton_generar, output)

In [ ]:
#ejemplo 4}
# Paso 1: Instalación de librerías necesarias
!pip install openpyxl

# Paso 2: Importaciones
import pandas as pd
import io
from google.colab import files
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import display
import ipywidgets as widgets

# Paso 3: Subida del archivo Excel
print("📤 Carga tu archivo Excel con el horario:")
uploaded = files.upload()

# Leer el archivo
for file_name in uploaded.keys():
    df = pd.read_excel(io.BytesIO(uploaded[file_name]), header=2)  # Cabecera en fila 3 (índice 2)

# Paso 4: Limpiar datos y transformar
dias_map = {
    'Lun': 'Lunes',
    'Mar': 'Martes',
    'Mier': 'Miércoles',
    'Jue': 'Jueves',
    'Vier': 'Viernes',
    'Sab': 'Sábado'
}

df = df[df['Asignatura'].notna()]  # Quitar filas vacías

df_largo = df.melt(
    id_vars=['Asignatura', 'Hora Ini', 'Hora Fin', 'Salon', 'Campus'],
    value_vars=dias_map.keys(),
    var_name='Día Abrev',
    value_name='Activo'
)

df_largo = df_largo[df_largo['Activo'] == 'Y']
df_largo['Día'] = df_largo['Día Abrev'].map(dias_map)
df_largo = df_largo.drop(columns=['Día Abrev', 'Activo'])

# Convertir horas a datetime para graficar
df_largo['Hora Ini'] = pd.to_datetime(df_largo['Hora Ini'].astype(str), format='%H:%M')
df_largo['Hora Fin'] = pd.to_datetime(df_largo['Hora Fin'].astype(str), format='%H:%M')

# Paso 5: Widget para seleccionar materias
materias = sorted(df_largo['Asignatura'].unique())

selector = widgets.SelectMultiple(
    options=materias,
    description='Materias:',
    layout=widgets.Layout(width='50%'),
    rows=10
)

display(selector)

# Paso 6: Función para graficar horario
def graficar_horario(df_filtrado):
    dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado']
    colores = plt.cm.tab20.colors

    fig, ax = plt.subplots(figsize=(12, 6))

    for i, dia in enumerate(dias):
        clases_dia = df_filtrado[df_filtrado['Día'] == dia]
        for j, (_, row) in enumerate(clases_dia.iterrows()):
            ini = row['Hora Ini']
            fin = row['Hora Fin']
            duracion = (fin - ini).seconds / 3600

            ax.broken_barh(
                [(ini.hour + ini.minute/60, duracion)],
                (i - 0.4, 0.8),
                facecolors=colores[j % len(colores)],
                edgecolors='black'
            )
            ax.text(ini.hour + ini.minute/60 + 0.05, i,
                    f"{row['Asignatura']} ({row['Salon']})",
                    va='center', ha='left', fontsize=8, color='black')

    ax.set_yticks(range(len(dias)))
    ax.set_yticklabels(dias)
    ax.set_xlabel("Hora del día")
    ax.set_xlim(7, 22)
    ax.grid(True, axis='x', linestyle='--', alpha=0.3)
    ax.set_title("📚 Horario de Clases")

    plt.tight_layout()
    plt.show()

# Paso 7: Botón para generar gráfica
boton = widgets.Button(description="Mostrar Horario", button_style='success')
output = widgets.Output()

def on_click(b):
    output.clear_output()
    with output:
        seleccionadas = list(selector.value)
        if not seleccionadas:
            print("⚠️ Selecciona al menos una materia.")
            return
        df_filtrado = df_largo[df_largo['Asignatura'].isin(seleccionadas)]
        graficar_horario(df_filtrado)

boton.on_click(on_click)
display(boton, output)


In [ ]:
# Ejemplo 5
# Código completo mejorado con guardado y visualización de cruces en Google Colab

# 📚 Importar librerías
from google.colab import files
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime

# 📂 Paso 1: Subir archivo Excel
uploaded = files.upload()
for nombre_archivo in uploaded.keys():
    df = pd.read_excel(io.BytesIO(uploaded[nombre_archivo]), header=2)

# 🧹 Paso 2: Limpiar y transformar los datos
df = df[df['Asignatura'].notna()]

dias_map = {
    'Lun': 'Lunes',
    'Mar': 'Martes',
    'Mier': 'Miércoles',
    'Jue': 'Jueves',
    'Vier': 'Viernes',
    'Sab': 'Sábado'
}

df_largo = df.melt(
    id_vars=['Asignatura', 'Nº Clase', 'Hora Ini', 'Hora Fin', 'Salon', 'Campus', 'Total Inscritos', 'Total Cupos'],
    value_vars=dias_map.keys(),
    var_name='Día Abrev',
    value_name='Activo'
)

df_largo = df_largo[df_largo['Activo'] == 'Y']
df_largo['Día'] = df_largo['Día Abrev'].map(dias_map)
df_largo = df_largo.drop(columns=['Día Abrev', 'Activo'])

df_largo['Total Inscritos'] = pd.to_numeric(df_largo['Total Inscritos'], errors='coerce')
df_largo['Total Cupos'] = pd.to_numeric(df_largo['Total Cupos'], errors='coerce')
df_largo['% Ocupación'] = (df_largo['Total Inscritos'] / df_largo['Total Cupos']) * 100

df_largo['Hora Ini'] = pd.to_datetime(df_largo['Hora Ini'].astype(str), format='%H:%M').dt.time
df_largo['Hora Fin'] = pd.to_datetime(df_largo['Hora Fin'].astype(str), format='%H:%M').dt.time

# 📋 Paso 3: Mostrar resumen general de materias
resumen = df_largo.groupby(['Asignatura', 'Nº Clase']).agg({
    'Día': lambda x: ', '.join(sorted(x.unique())),
    'Hora Ini': lambda x: min(x).strftime('%H:%M'),
    'Hora Fin': lambda x: max(x).strftime('%H:%M'),
    'Total Inscritos': 'first',
    'Total Cupos': 'first',
    '% Ocupación': 'first'
}).reset_index()

print("📋 Oferta académica disponible:")
display(resumen)

# ✅ Paso 4: Widgets para seleccionar materias y botones guardar y mostrar cruces
materias = sorted(df_largo['Asignatura'].unique())
selector_materias = widgets.SelectMultiple(
    options=materias,
    description='Materias:',
    layout=widgets.Layout(width='50%'),
    rows=10
)

boton_generar = widgets.Button(description="Generar horario", button_style='success')
boton_guardar = widgets.Button(description="Guardar horario seleccionado", button_style='info')
boton_mostrar_cruces = widgets.Button(description="Mostrar cruces horarios guardados", button_style='warning')
output = widgets.Output()
output_guardar = widgets.Output()
output_cruces = widgets.Output()

# Variable global para guardar horarios seleccionados
horario_guardado = pd.DataFrame(columns=df_largo.columns)

# 🗓️ Funciones de visualización y utilidades
def hora_a_minutos(t):
    return t.hour * 60 + t.minute

def color_por_ocupacion(pct):
    if pct < 50:
        return 'green'
    elif pct <= 90:
        return 'gold'
    else:
        return 'red'

dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado']
mapa_dias = {dia: i for i, dia in enumerate(dias_semana)}

def mostrar_calendario(horario_df, titulo="🗓️ Horario personalizado del estudiante"):
    fig, ax = plt.subplots(figsize=(14, 8))

    for _, row in horario_df.iterrows():
        dia_idx = mapa_dias[row['Día']]
        ini_min = hora_a_minutos(row['Hora Ini'])
        fin_min = hora_a_minutos(row['Hora Fin'])
        duracion = fin_min - ini_min

        rect = patches.Rectangle(
            (dia_idx, ini_min),
            width=0.9,
            height=duracion,
            facecolor=color_por_ocupacion(row['% Ocupación']),
            edgecolor='black',
            alpha=0.8
        )
        ax.add_patch(rect)
        ax.text(
            dia_idx + 0.05, ini_min + duracion / 2,
            f"{row['Asignatura']}\nClase #{int(row['Nº Clase'])}\n{row['Hora Ini'].strftime('%H:%M')} - {row['Hora Fin'].strftime('%H:%M')}",
            fontsize=8,
            verticalalignment='center',
            color='black',
            wrap=True
        )

    ax.set_xlim(0, len(dias_semana))
    ax.set_ylim(1320, 360)  # Invertido: empieza arriba en 6:00 (360) y termina abajo en 22:00 (1320)
    ax.set_xticks(range(len(dias_semana)))
    ax.set_xticklabels(dias_semana)
    ax.set_yticks(range(420, 1321, 60))
    ax.set_yticklabels([f"{h}:00" for h in range(7, 23)])
    ax.set_xlabel("Día")
    ax.set_ylabel("Hora")
    ax.set_title(titulo)
    ax.grid(True, axis='y', linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

def on_boton_generar_clicked(b):
    with output:
        output.clear_output(wait=True)
        seleccion = list(selector_materias.value)
        if not seleccion:
            print("⚠️ Por favor selecciona al menos una materia.")
            return
        df_seleccion = df_largo[df_largo['Asignatura'].isin(seleccion)]
        mostrar_calendario(df_seleccion)

def on_boton_guardar_clicked(b):
    global horario_guardado
    with output_guardar:
        output_guardar.clear_output(wait=True)
        seleccion = list(selector_materias.value)
        if not seleccion:
            print("⚠️ Por favor selecciona al menos una materia para guardar.")
            return
        df_seleccion = df_largo[df_largo['Asignatura'].isin(seleccion)]

        # Evitar duplicados agregando solo si no están ya en el guardado
        # Se puede hacer por combinación de columnas clave
        keys_guardado = horario_guardado[['Asignatura', 'Nº Clase', 'Día', 'Hora Ini', 'Hora Fin']]
        keys_nuevos = df_seleccion[['Asignatura', 'Nº Clase', 'Día', 'Hora Ini', 'Hora Fin']]
        df_nuevos_unicos = df_seleccion[~keys_nuevos.apply(tuple,1).isin(keys_guardado.apply(tuple,1))]

        if df_nuevos_unicos.empty:
            print("ℹ️ Todas las materias seleccionadas ya están guardadas.")
            return
        horario_guardado = pd.concat([horario_guardado, df_nuevos_unicos], ignore_index=True)
        print(f"✅ Horario de {len(df_nuevos_unicos['Asignatura'].unique())} materias guardado correctamente.")

def mostrar_cruces_horarios(horario_df):
    # Visualizar todas las materias guardadas y marcar cruces de horarios
    fig, ax = plt.subplots(figsize=(14, 8))

    if horario_df.empty:
        print("⚠️ No hay horarios guardados para mostrar.")
        return

    # Pintar todos los bloques
    for _, row in horario_df.iterrows():
        dia_idx = mapa_dias[row['Día']]
        ini_min = hora_a_minutos(row['Hora Ini'])
        fin_min = hora_a_minutos(row['Hora Fin'])
        duracion = fin_min - ini_min

        ax.add_patch(
            patches.Rectangle(
                (dia_idx, ini_min),
                width=0.9,
                height=duracion,
                facecolor=color_por_ocupacion(row['% Ocupación']),
                edgecolor='black',
                alpha=0.5
            )
        )
        ax.text(
            dia_idx + 0.05, ini_min + duracion / 2,
            f"{row['Asignatura']}\nClase #{int(row['Nº Clase'])}\n{row['Hora Ini'].strftime('%H:%M')} - {row['Hora Fin'].strftime('%H:%M')}",
            fontsize=8,
            verticalalignment='center',
            color='black',
            wrap=True
        )

    # Detectar cruces por día
    for dia in dias_semana:
        df_dia = horario_df[horario_df['Día'] == dia]
        # Ordenar por hora inicio
        df_dia = df_dia.sort_values(by='Hora Ini')

        # Detectar solapamiento para pares
        intervals = []
        for _, row in df_dia.iterrows():
            intervals.append((hora_a_minutos(row['Hora Ini']), hora_a_minutos(row['Hora Fin']), row['Asignatura'], row['Nº Clase']))
        # Comparar pares para detectar solapamientos
        for i in range(len(intervals)):
            for j in range(i+1, len(intervals)):
                start_i, end_i, mat_i, clase_i = intervals[i]
                start_j, end_j, mat_j, clase_j = intervals[j]
                # Si se solapan
                if start_j < end_i and start_i < end_j:
                    solap_min_start = max(start_i, start_j)
                    solap_min_end = min(end_i, end_j)
                    dur_solap = solap_min_end - solap_min_start
                    dia_idx = mapa_dias[dia]
                    # Pintar rectángulo rojo translúcido indicando el cruce
                    ax.add_patch(
                        patches.Rectangle(
                            (dia_idx, solap_min_start),
                            width=0.9,
                            height=dur_solap,
                            facecolor='red',
                            edgecolor='darkred',
                            alpha=0.7
                        )
                    )
                    # Opcional: texto del solapamiento más visible o no
                    ax.text(
                        dia_idx + 0.45, solap_min_start + dur_solap / 2,
                        f"Clases cruzadas:\n{mat_i} - {mat_j}",
                        fontsize=7,
                        color='white',
                        verticalalignment='center',
                        horizontalalignment='center',
                        weight='bold'
                    )

    ax.set_xlim(0, len(dias_semana))
    ax.set_ylim(1320, 360)
    ax.set_xticks(range(len(dias_semana)))
    ax.set_xticklabels(dias_semana)
    ax.set_yticks(range(420, 1321, 60))
    ax.set_yticklabels([f"{h}:00" for h in range(7, 23)])
    ax.set_xlabel("Día")
    ax.set_ylabel("Hora")
    ax.set_title("⚠️ Visualización de cruces de horarios guardados")
    ax.grid(True, axis='y', linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

def on_boton_mostrar_cruces_clicked(b):
    with output_cruces:
        output_cruces.clear_output(wait=True)
        if horario_guardado.empty:
            print("⚠️ No hay horarios guardados para mostrar. Guarda al menos una materia primero.")
            return
        mostrar_cruces_horarios(horario_guardado)

# Asignar funciones a botones
boton_generar.on_click(on_boton_generar_clicked)
boton_guardar.on_click(on_boton_guardar_clicked)
boton_mostrar_cruces.on_click(on_boton_mostrar_cruces_clicked)

# Mostrar widgets y salidas
display(selector_materias, widgets.HBox([boton_generar, boton_guardar, boton_mostrar_cruces]),
        output, output_guardar, output_cruces)



In [1]:
#dsadsaf
# 📚 Importar librerías
from google.colab import files
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime
import itertools

# 📂 Paso 1: Subir archivos Excel
uploaded = files.upload()

# Leer todos los archivos y combinarlos
frames = []
for nombre_archivo in uploaded.keys():
    df_temp = pd.read_excel(io.BytesIO(uploaded[nombre_archivo]), header=2)
    frames.append(df_temp)
df = pd.concat(frames, ignore_index=True)

# 🧹 Paso 2: Limpiar y transformar los datos
df = df[df['Asignatura'].notna()]

dias_map = {
    'Lun': 'Lunes',
    'Mar': 'Martes',
    'Mier': 'Miércoles',
    'Jue': 'Jueves',
    'Vier': 'Viernes',
    'Sab': 'Sábado'
}

df_largo = df.melt(
    id_vars=['Asignatura', 'Nº Clase', 'Hora Ini', 'Hora Fin', 'Salon', 'Campus', 'Total Inscritos', 'Total Cupos'],
    value_vars=dias_map.keys(),
    var_name='Día Abrev',
    value_name='Activo'
)

df_largo = df_largo[df_largo['Activo'] == 'Y']
df_largo['Día'] = df_largo['Día Abrev'].map(dias_map)
df_largo = df_largo.drop(columns=['Día Abrev', 'Activo'])

df_largo['Total Inscritos'] = pd.to_numeric(df_largo['Total Inscritos'], errors='coerce')
df_largo['Total Cupos'] = pd.to_numeric(df_largo['Total Cupos'], errors='coerce')
df_largo['% Ocupación'] = (df_largo['Total Inscritos'] / df_largo['Total Cupos']) * 100

df_largo['Hora Ini'] = pd.to_datetime(df_largo['Hora Ini'].astype(str), format='%H:%M').dt.time
df_largo['Hora Fin'] = pd.to_datetime(df_largo['Hora Fin'].astype(str), format='%H:%M').dt.time

# 📋 Paso 3: Mostrar resumen general de materias
resumen = df_largo.groupby(['Asignatura', 'Nº Clase']).agg({
    'Día': lambda x: ', '.join(sorted(x.unique())),
    'Hora Ini': lambda x: min(x).strftime('%H:%M'),
    'Hora Fin': lambda x: max(x).strftime('%H:%M'),
    'Total Inscritos': 'first',
    'Total Cupos': 'first',
    '% Ocupación': 'first'
}).reset_index()

print("\U0001F4CB Oferta académica disponible:")
display(resumen)

# ✅ Paso 4: Widgets de selección con filtro de materias
materias = sorted(df_largo['Asignatura'].unique())

busqueda_materias = widgets.Text(
    description='Buscar:',
    placeholder='Escribe para filtrar materias',
    layout=widgets.Layout(width='50%')
)

selector_materias = widgets.SelectMultiple(
    options=materias,
    description='Materias:',
    layout=widgets.Layout(width='50%'),
    rows=12
)

def filtrar_materias_por_texto(cambio):
    texto = cambio['new'].lower()
    opciones_filtradas = [m for m in materias if texto in m.lower()]
    selector_materias.options = opciones_filtradas

busqueda_materias.observe(filtrar_materias_por_texto, names='value')

selector_jornada = widgets.RadioButtons(
    options=['Mañana (6:00 - 14:00)', 'Noche (18:00 - 22:00)', 'Mixta (6:00 - 22:00)'],
    description='Jornada:',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

# 🏫 Selector de sede
selector_sede = widgets.RadioButtons(
    options=['Todas', 'Chapinero', 'Sur', 'Crisanto Luque'],
    description='Sede:',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

boton_generar = widgets.Button(description="Generar horario", button_style='success')
boton_anterior = widgets.Button(description="Anterior", button_style='info')
boton_siguiente = widgets.Button(description="Siguiente", button_style='info')
output = widgets.Output()
nav_box = widgets.HBox([boton_anterior, boton_siguiente])

# 🗓️ Paso 5: Visualización
def hora_a_minutos(t):
    return t.hour * 60 + t.minute

def color_por_ocupacion(pct):
    if pct < 50:
        return 'green'
    elif pct <= 90:
        return 'gold'
    else:
        return 'red'

dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado']
mapa_dias = {dia: i for i, dia in enumerate(dias_semana)}

def mostrar_calendario(horario_df):
    fig, ax = plt.subplots(figsize=(14, 8))
    for _, row in horario_df.iterrows():
        dia_idx = mapa_dias[row['Día']]
        ini_min = hora_a_minutos(row['Hora Ini'])
        fin_min = hora_a_minutos(row['Hora Fin'])
        duracion = fin_min - ini_min

        rect = patches.Rectangle(
            (dia_idx, ini_min),
            width=0.9,
            height=duracion,
            facecolor=color_por_ocupacion(row['% Ocupación']),
            edgecolor='black',
            alpha=0.8
        )
        ax.add_patch(rect)
        ax.text(
            dia_idx + 0.05, ini_min + duracion / 2,
            f"{row['Asignatura']}\nClase #{int(row['Nº Clase'])}\n{row['Hora Ini'].strftime('%H:%M')} - {row['Hora Fin'].strftime('%H:%M')}",
            fontsize=8,
            verticalalignment='center',
            color='black'
        )

    ax.set_xlim(0, len(dias_semana))
    ax.set_ylim(1320, 360)
    ax.set_xticks(range(len(dias_semana)))
    ax.set_xticklabels(dias_semana)
    ax.set_yticks(range(360, 1321, 60))
    ax.set_yticklabels([f"{h}:00" for h in range(6, 23)])
    ax.set_xlabel("Día")
    ax.set_ylabel("Hora")
    ax.set_title("\U0001F5D3️ Horario personalizado del estudiante")
    ax.grid(True, axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

# 🧠 Auxiliares
def clases_se_solan(c1, c2):
    if c1['Día'] != c2['Día']:
        return False
    ini1, fin1 = hora_a_minutos(c1['Hora Ini']), hora_a_minutos(c1['Hora Fin'])
    ini2, fin2 = hora_a_minutos(c2['Hora Ini']), hora_a_minutos(c2['Hora Fin'])
    return max(ini1, ini2) < min(fin1, fin2)

def combinaciones_validas(opciones_por_materia):
    combinaciones = []
    for combinacion in itertools.product(*opciones_por_materia):
        solapamiento = False
        for i in range(len(combinacion)):
            for j in range(i + 1, len(combinacion)):
                if any(clases_se_solan(c1, c2) for _, c1 in combinacion[i].iterrows() for _, c2 in combinacion[j].iterrows()):
                    solapamiento = True
                    break
            if solapamiento:
                break
        if not solapamiento:
            combinaciones.append(pd.concat(combinacion))
    return combinaciones

# ▶️ Paso 6: Generar combinaciones según filtros
combinaciones_filtradas = []
indice_actual = 0

def on_boton_generar_clicked(b):
    global combinaciones_filtradas, indice_actual
    with output:
        clear_output(wait=True)
        seleccion = list(selector_materias.value)
        jornada = selector_jornada.value
        sede = selector_sede.value

        if not seleccion:
            print("⚠️ Por favor selecciona al menos una materia.")
            return

        # Filtrado inicial: materias seleccionadas y con cupo disponible
        df_filtrado = df_largo[
            (df_largo['Asignatura'].isin(seleccion)) &
            (df_largo['Total Inscritos'] < df_largo['Total Cupos'])
        ]

        # Filtrar por sede
        if sede == 'Sur':
            df_filtrado = df_filtrado[df_filtrado['Salon'].str.startswith('SUR', na=False)]
        elif sede == 'Crisanto Luque':
            df_filtrado = df_filtrado[df_filtrado['Salon'].str.contains('SLUQ', na=False)]
        elif sede == 'Chapinero':
            df_filtrado = df_filtrado[
                ~df_filtrado['Salon'].str.startswith('SUR', na=False) &
                ~df_filtrado['Salon'].str.contains('SLUQ', na=False)
            ]

        # Verificar si cada materia está presente en la sede seleccionada
        materias_disponibles = df_filtrado['Asignatura'].unique()
        materias_no_disponibles = [m for m in seleccion if m not in materias_disponibles]

        if materias_no_disponibles:
            if sede == 'Sur':
                print(f"⚠️ No hay opciones de esta combinación para la sede Sur. Materias no disponibles: {', '.join(materias_no_disponibles)}")
            elif sede == 'Crisanto Luque':
                print(f"⚠️ No hay opciones de esta combinación para la sede Crisanto Luque. Materias no disponibles: {', '.join(materias_no_disponibles)}")
            else:
                print(f"⚠️ No hay opciones de esta combinación para la sede {sede}. Materias no disponibles: {', '.join(materias_no_disponibles)}")
            return

        if df_filtrado.empty:
            print("⚠️ No hay clases disponibles para esta combinación de filtros.")
            return

        if 'Mañana' in jornada:
            hora_ini_limite = 6 * 60
            hora_fin_limite = 14 * 60
        elif 'Noche' in jornada:
            hora_ini_limite = 18 * 60
            hora_fin_limite = 22 * 60
        else:
            hora_ini_limite = 6 * 60
            hora_fin_limite = 22 * 60

        opciones_por_materia = []
        for materia in seleccion:
            clases = df_filtrado[df_filtrado['Asignatura'] == materia]
            clases_por_num = [grupo for _, grupo in clases.groupby('Nº Clase')]
            if clases_por_num:
                opciones_por_materia.append(clases_por_num)

        if not opciones_por_materia:
            print("⚠️ No hay opciones válidas para generar combinaciones.")
            return

        combinaciones = combinaciones_validas(opciones_por_materia)

        combinaciones_filtradas = []
        for comb in combinaciones:
            if all(hora_ini_limite <= hora_a_minutos(h) <= hora_fin_limite for h in comb['Hora Ini']) and \
               all(hora_ini_limite <= hora_a_minutos(h) <= hora_fin_limite for h in comb['Hora Fin']):
                combinaciones_filtradas.append(comb)

        if not combinaciones_filtradas:
            print("⚠️ No hay opciones para esta jornada.")
            return

        indice_actual = 0
        mostrar_combinacion_actual()

def mostrar_combinacion_actual():
    with output:
        clear_output(wait=True)
        print(f"✅ Mostrando combinación #{indice_actual + 1} de {len(combinaciones_filtradas)}")
        comb = combinaciones_filtradas[indice_actual]
        display(comb[['Asignatura', 'Nº Clase', 'Día', 'Hora Ini', 'Hora Fin', 'Salon']])
        mostrar_calendario(comb)

def on_boton_anterior_clicked(b):
    global indice_actual
    if combinaciones_filtradas:
        indice_actual = (indice_actual - 1) % len(combinaciones_filtradas)
        mostrar_combinacion_actual()

def on_boton_siguiente_clicked(b):
    global indice_actual
    if combinaciones_filtradas:
        indice_actual = (indice_actual + 1) % len(combinaciones_filtradas)
        mostrar_combinacion_actual()

# ▶️ Conectar eventos
boton_generar.on_click(on_boton_generar_clicked)
boton_anterior.on_click(on_boton_anterior_clicked)
boton_siguiente.on_click(on_boton_siguiente_clicked)

# ▶️ Mostrar UI
display(widgets.VBox([
    busqueda_materias,
    selector_materias,
    selector_jornada,
    selector_sede,
    boton_generar,
    nav_box,
    output
]))

Saving BOGOTA_PRIME.xlsx to BOGOTA_PRIME.xlsx
📋 Oferta académica disponible:


,Asignatura,Nº Clase,Día,Hora Ini,Hora Fin,Total Inscritos,Total Cupos,% Ocupación
0,ANÁLISIS DE SEÑALES,2929,"Jueves, Martes",18:00,22:00,27,27,100.000000
1,ANÁLISIS DE SEÑALES,2937,"Lunes, Miércoles",18:00,22:00,29,27,107.407407
2,ANÁLISIS DE SEÑALES,4931,"Miércoles, Viernes",08:00,12:00,19,35,54.285714
3,ANÁLISIS DE SEÑALES,5338,"Jueves, Martes",18:00,22:00,27,30,90.000000
4,ANÁLISIS DE SEÑALES,5339,"Lunes, Miércoles",18:00,22:00,30,30,100.000000
...,...,...,...,...,...,...,...,...
60,SISTEMAS DINÁMICOS,3062,"Lunes, Viernes",18:00,20:00,33,34,97.058824
61,SISTEMAS DINÁMICOS,5340,"Lunes, Viernes",18:00,20:00,27,35,77.142857
62,Sistemas Embebidos,2975,"Sábado, Viernes",08:00,20:00,20,27,74.074074
63,Sistemas Embebidos,2995,"Lunes, Miércoles",20:00,22:00,22,27,81.481481
